In [ ]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import pandas as pd
import spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 20.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Cargar los datos de entrenamiento
data = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', usecols=[0, 1, 2, 3], engine='python')

# Cargar el modelo de spaCy para tokenizar oraciones
nlp = spacy.load("es_core_news_sm")

In [ ]:
# Función para obtener los enunciados con las keywords
def extract_keyword_sentences(row):
    keywords = set(row['keyword'].split(', '))  # Convertir las keywords en un conjunto para eliminar duplicados
    text = row['text']
    doc = nlp(text)
    found_sentences = set()  # Conjunto para almacenar las oraciones encontradas
    for sentence in doc.sents:
        # Verificar si la oración contiene al menos una de las keywords
        if any(keyword.lower() in sentence.text.lower() for keyword in keywords):
            found_sentences.add(sentence.text.strip())  # Agregar la oración al conjunto de oraciones encontradas
    return '. '.join(found_sentences) if found_sentences else None  # Unir las oraciones encontradas en un solo texto

In [ ]:
# Filtrar y actualizar el texto con los enunciados que contienen las keywords
data['text'] = data.apply(lambda row: extract_keyword_sentences(row), axis=1)
datatext = data['text'].astype(str)
print(len(datatext))
print(datatext[0:10])

600
0    Yesterday I went on a party cruise alone and m...
1    I run a game server and I really love doing so...
2    mini golf?).. If you want a long-running relat...
3    I kind of accidentally psyched myself out of t...
4                  I’ve been in negative waves lately.
5    nnThe reality of our landscape does not change...
6    Sorry if I'm sounding too crazy, I was just ha...
7    Because I've been "forced" (by the nature of m...
8    I was terrible at Socializing with new people ...
9            Have I run out of things to say, perhaps?
Name: text, dtype: object


In [ ]:
# Convertir cada texto en el formato deseado
formatted_texts = datatext.apply(lambda text: f"('{text}')")
print(len(formatted_texts))

600




---



## **MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli**


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_6 = []
for text in formatted_texts:
    output_6 = classifier(text, candidate_labels=['related to an outdoor space', 'not related to an outdoor space'])
    predictions_6.append(output_6)
    print(output_6)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': "('Yesterday I went on a party cruise alone and made friends.')", 'labels': ['related to an outdoor space', 'not related to an outdoor space'], 'scores': [0.9467911720275879, 0.053208865225315094]}
{'sequence': "('I run a game server and I really love doing so.. I shouldn't have to say it on this sub, but I'm not looking for hit-and-runners, I'm sick of them.. Always tried to run one when I could.')", 'labels': ['not related to an outdoor space', 'related to an outdoor space'], 'scores': [0.9990729689598083, 0.0009270601440221071]}
{'sequence': "('mini golf?).. If you want a long-running relationship, it will come up at some point anyway.')", 'labels': ['related to an outdoor space', 'not related to an outdoor space'], 'scores': [0.8952968716621399, 0.10470309853553772]}
{'sequence': "('I kind of accidentally psyched myself out of thinking I was sea-bottom scum.')", 'labels': ['not related to an outdoor space', 'related to an outdoor space'], 'scores': [0.7369334101676941,

In [ ]:
# Contar el total de clasificaciones como 'related to a natural outdoor space' y 'not related to a natural outdoor space'
total_outdoor_space = sum(1 for pred in predictions_6 if pred['labels'][0] == 'related to an outdoor space')
total_not_outdoor_space = sum(1 for pred in predictions_6 if pred['labels'][0] == 'not related to an outdoor space')

# Imprimir los resultados
print("Total de clasificados como 'related to an outdoor space':", total_outdoor_space)
print("Total de clasificados como 'not related to an outdoor space':", total_not_outdoor_space)

Total de clasificados como 'related to an outdoor space': 319
Total de clasificados como 'not related to an outdoor space': 281


In [ ]:
from sklearn.metrics import classification_report

# Reemplazar los valores de ña columna 'label' que no son 0 por 1
data['Classification'] = data['Classification'].apply(lambda x: 1 if x !=0 else x)

# Obtener las etiquetas reales del conjunto de datos
y_true = data['Classification']

# Convertir las etiquetas predichas a números según el formato especificado en tu pregunta
label_map = {
    'related to an outdoor space': 1,
    'not related to an outdoor space': 0
}
# Mapear las etiquetas predichas al formato numérico
y_pred = [label_map[prediction['labels'][0]] for prediction in predictions_6]

# Obtener el classification report
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.88      0.66      0.75       377
           1       0.59      0.85      0.70       223

    accuracy                           0.73       600
   macro avg       0.74      0.75      0.72       600
weighted avg       0.77      0.73      0.73       600

